Run the line below (without the # symbol) to install pandas if not already installed

In [60]:
# !py -m pip install pandas

In [61]:
import requests
import csv
import os
import pandas as pd
import json
import re
import time
from datetime import datetime, timedelta
import math
import traceback

In [62]:
input_path = "C:\\Users\\khan32\\Documents\\factgrid_py"
# input_path = "."

In [63]:
output_path = input_path

In [64]:
place_name = "Bamberg"

In [65]:
today_string = datetime.now().strftime('%Y-%m-%d')

In [66]:
# SELECT * FROM `role` r INNER JOIN url_external u on r.id = u.item_id where u.authority_id = 42 

Export the results from the query 
```sql
SELECT * FROM role r INNER JOIN url_external u on r.id = u.item_id where u.authority_id = 42
```  
from the main wiag database in the csv format.

Rename it to include the date. An example filename would be `role_2024_04_24.csv`

In [69]:
input_file = f"role_2024_05_14.csv"
input_path_file = os.path.join(input_path, input_file)
wiag_roles_df = pd.read_csv(input_path_file, names=['id', 'note', 'name', 'comment', 'gs_reg_id', 'generic_term', 'plural', 'definition', 'role_gp_fg_id', 'gender', 'lang', 'role_group_id','r_id','item_id','r_note','authority_id','role_fg_id','r_comment'])
len(wiag_roles_df)

102

### Download data from wiag
https://wiag-vokabulare.uni-goettingen.de/query/can

In [70]:
input_file = f"WIAG-Domherren-DB-Ämter_2024-05-14_Bamberg.csv"
input_path_file = os.path.join(input_path, input_file)
role_all_df = pd.read_csv(input_path_file, sep=';')
len(role_all_df)

2798

In [71]:
last_modified = datetime.fromtimestamp(os.path.getmtime(input_file))
now = datetime.now()
assert last_modified.day == now.day and last_modified.month == now.month, f"The file was last updated on {last_modified.strftime('%d.%m')}"

### ERROR: If you get an error when you run the line above this means that the file was not updated today. 
A few solutions: 
* update the file again by downloading it again
* change the file name to something correct
* (not recommended) continue if you are sure that you need to use old data.

In [72]:
len(role_all_df)

2798

In [73]:
role_all_df.head()

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702


# Download data from factgrid

If any of the following requests to factgrid fail, try re running the cells.

In [74]:
url = 'https://database.factgrid.de/sparql'
query = (
    """SELECT ?item ?gsn WHERE {
  ?item wdt:P471 ?gsn
}
"""
)
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# make request: 
r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_institution_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_institution_df)

268

In [75]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?item ?wiagid ?label ?alternative WHERE {
  ?item wdt:P2/wdt:P3* wd:Q164535.
  #?item schema:description ?itemDesc.
  ?item rdfs:label ?label.
  OPTIONAL {?item schema:description ?itemDesc.}
  OPTIONAL {?item skos:altLabel ?alternative. }
  OPTIONAL {?item wdt:P601 ?wiagid.}
  FILTER(LANG(?label) in ("en", "de"))
}
"""
)


# version 2
# SELECT ?item ?wiagid (group_concat(DISTINCT ?label; separator=',') as ?labels) (group_concat(DISTINCT ?itemDesc; separator=',') as ?itemDescs) (group_concat(DISTINCT ?alternative ; separator=',') as ?alternatives) WHERE {
#   ?item wdt:P2/wdt:P3* wd:Q164535.
#   ?item schema:description ?itemDesc.
#   ?item rdfs:label ?label.
#   OPTIONAL {?item schema:description ?itemDesc.}
#   OPTIONAL {?item skos:altLabel ?alternative. }
#   OPTIONAL {?item wdt:P601 ?wiagid.}
#   FILTER(LANG(?label) in ("en", "de"))
# }
# GROUP BY ?item ?wiagid

# SELECT ?item ?wiagid WHERE {
#   ?item wdt:P2/wdt:P3* wd:Q164535.
#   ?item wdt:P601 ?wiagid
# }
# SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# make request: 
r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_diocese_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_diocese_df)

1290

In [76]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?item ?label WHERE {
  ?item wdt:P2 wd:Q257052.
  ?item rdfs:label ?label.
  FILTER(LANG(?label) in ("de"))
}
"""
)

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
factgrid_inst_roles_df = pd.json_normalize(data['results']['bindings'])

len(factgrid_inst_roles_df)

4343

# Clean Factgrid data

In [77]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [78]:
extract_qid(factgrid_institution_df, 'item.value')
extract_qid(factgrid_diocese_df, 'item.value')
extract_qid(factgrid_inst_roles_df, 'item.value')

In [79]:
drop_type_columns(factgrid_institution_df)
drop_type_columns(factgrid_diocese_df)
drop_type_columns(factgrid_inst_roles_df)

In [80]:
# rename columns
factgrid_institution_df.columns = ['fg_institution_id', 'fg_gsn_id']
factgrid_diocese_df.columns = ["fg_diocese_id", "dioc_label", "dioc_alt", "dioc_wiag_id"]
factgrid_inst_roles_df.columns = ["fg_inst_role_id", "inst_role"]

In [81]:
# clean the diocese alts by removing BITECA and BETA entries 
factgrid_diocese_df['dioc_alt'] = factgrid_diocese_df['dioc_alt'].replace(['^BITECA.*', '^BETA.*'], '', regex=True)
factgrid_diocese_df
# set(factgrid_diocese_df['fg_alts'])

,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,Q153265,Bistum Trient,Diözese Trient,WIAG-Inst-DIOCGatz-069-001
1,Q153265,Diocese of Trento,Diözese Trient,WIAG-Inst-DIOCGatz-069-001
2,Q153266,Bistum Utrecht,Diözese Utrecht,WIAG-Inst-DIOCGatz-070-001
3,Q153266,Diocese of Utrecht,Diözese Utrecht,WIAG-Inst-DIOCGatz-070-001
4,Q153267,Bistum Wien,Diözese Wien,WIAG-Inst-DIOCGatz-071-001
...,...,...,...,...
1285,Q257206,diocese of Kraków,NaN,NaN
1286,Q395086,Erzbistum Sevilla,NaN,NaN
1287,Q395086,Archdiocese of Seville,NaN,NaN
1288,Q395086,Erzbistum Sevilla,NaN,NaN


In [82]:
factgrid_institution_df['fg_gsn_id'] = pd.to_numeric(factgrid_institution_df['fg_gsn_id'], downcast='float')

In [83]:
fg_gp = factgrid_institution_df.groupby('fg_gsn_id').count()
duplicate_fg_entries = factgrid_institution_df[factgrid_institution_df['fg_gsn_id'].isin(list(fg_gp[fg_gp['fg_institution_id'] > 1].index))]
assert duplicate_fg_entries.empty, f"There are possible duplicates on factgrid\n.{duplicate_fg_entries}"

In [84]:
# factgrid_institution_df.drop_duplicates(['fg_gsn_id'], inplace=True)

In [85]:
factgrid_institution_df.dtypes

fg_institution_id     object
fg_gsn_id            float32
dtype: object

In [86]:
factgrid_diocese_df.dtypes

fg_diocese_id    object
dioc_label       object
dioc_alt         object
dioc_wiag_id     object
dtype: object

In [87]:
role_all_df.dtypes

person_id            object
id                    int64
name                 object
role_group           object
role_group_en        object
role_group_fq_id     object
institution          object
institution_id      float64
diocese              object
diocese_id           object
date_begin           object
date_end             object
date_sort_key         int64
GND                  object
GSN                  object
FactGrid             object
dtype: object

In [88]:
# TODO: Find a way to query amter data directly from wiag
# url = 'https://wiag-vocab.adw-goe.de/domherr/data'
# r = requests.get(url, params={'domstift': place_name})
# data = r.json()
# person_import_df = pd.json_normalize(data['persons'])

# print(len(person_import_df))
# person_import_df.head()

In [89]:
factgrid_institution_df.head()

,fg_institution_id,fg_gsn_id
0,Q422286,114.0
1,Q470546,119.0
2,Q633292,120.0
3,Q633339,131.0
4,Q633346,139.0


In [90]:
factgrid_diocese_df.head()

,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,Q153265,Bistum Trient,Diözese Trient,WIAG-Inst-DIOCGatz-069-001
1,Q153265,Diocese of Trento,Diözese Trient,WIAG-Inst-DIOCGatz-069-001
2,Q153266,Bistum Utrecht,Diözese Utrecht,WIAG-Inst-DIOCGatz-070-001
3,Q153266,Diocese of Utrecht,Diözese Utrecht,WIAG-Inst-DIOCGatz-070-001
4,Q153267,Bistum Wien,Diözese Wien,WIAG-Inst-DIOCGatz-071-001


# Check for missing data

## Check for missing institutions

In [91]:
missing_institution_on_factgrid_df = role_all_df.merge(
    factgrid_institution_df, indicator = True, how='left', left_on='institution_id', right_on='fg_gsn_id', suffixes=('_wiag', '_institute_fg')
).loc[lambda x : x['_merge']!='both']

In [92]:
print(len(missing_institution_on_factgrid_df))
missing_institution_on_factgrid_df.head()

257


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,_merge
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844,NaN,NaN,left_only
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691,NaN,NaN,left_only
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702,NaN,NaN,left_only
7,WIAG-Pers-EPISCGatz-04423-001,4940,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Mainz,WIAG-Inst-DIOCGatz-016-001,1051,1059,1051150,136648185,007-02348-001,Q653060,NaN,NaN,left_only
9,WIAG-Pers-EPISCGatz-05263-001,82446,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,1085,1088,1085150,119063085,009-01109-001,Q656048,NaN,NaN,left_only


In [93]:
# missing_institution_on_factgrid_df[['institution', 'institution_id', 'diocese', 'diocese_id']]

## Create a csv file with persons having no institution_id, diocese_id and diocese

In [94]:
missing_institution_diocese_on_factgrid_df = missing_institution_on_factgrid_df.merge(
    factgrid_diocese_df, indicator = '_second_merge', how='left', left_on='diocese_id', right_on='dioc_wiag_id', suffixes=('_wiag', '_dioc_fg')
).loc[lambda x : x['_second_merge']!='both']
missing_institution_diocese_on_factgrid_df.drop(['_merge','_second_merge'], axis=1, inplace=True)
null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df[['institution_id', 'diocese_id', 'diocese']].isna().all(axis=1)]
null_entries = null_entries[null_entries['name'] != 'Kardinal'].sort_values('person_id')

# 4312 to 7979 should be removed
null_entries = null_entries[null_entries.person_id.str.contains('[0-7][0-9]{4}-[0-9]{3}$', regex=True)]
null_entries.to_csv(f'simple_null_entries_amter-{today_string}.csv', sep=';')
assert len(null_entries) == 0

## Add diocese information from factgrid

Diocese is matched by 3 queries between wiag and factgrid. They are applied in the following priority. If a higher priority query find a match, the following query does nothing:
* wiag id (wiag) => wiag id (factgrid)
* diocese name (wiag) => diocese label (factgrid)
* diocese name (wiag) => diocese alt label (factgrid)

In [95]:
# lookup for the diocese by the diocese_id, then diocese labels, then diocese alts
def join_dioceses(input_df):
    rows = []
    for _, row in input_df.iterrows():
        search_row = pd.Series({'fg_diocese_id':None, 'dioc_label':None, 'dioc_alt':None, 'dioc_wiag_id':None})

        # NOTE: in the following three code blocks only the query changes.
        # this could be refactored somehow without running the query when defining it
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_wiag_id'] == row['diocese_id']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
    
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_label'] == row['diocese']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
    
        if not search_row['fg_diocese_id']:
            query = factgrid_diocese_df[factgrid_diocese_df['dioc_alt'] == row['diocese']]
            if not query.empty:
                search_row = query.head(1).squeeze(axis=0)
        rows.append(row.combine_first(search_row))
    
    dioc_joined_df = pd.DataFrame(rows)
    dioc_joined_df = dioc_joined_df[[*input_df.columns, *factgrid_diocese_df.columns]]
    
    return dioc_joined_df
dioc_joined_df = join_dioceses(missing_institution_on_factgrid_df)
print(len(dioc_joined_df))
dioc_joined_df.head()

257


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,_merge,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,120912619,062-01186-001,Q653844,NaN,NaN,left_only,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,136969143,059-00710-001,Q652691,NaN,NaN,left_only,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,118699334,006-00007-001,Q652702,NaN,NaN,left_only,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001
7,WIAG-Pers-EPISCGatz-04423-001,4940,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Mainz,WIAG-Inst-DIOCGatz-016-001,...,136648185,007-02348-001,Q653060,NaN,NaN,left_only,Q153230,Erzbistum Mainz,Bistum Mainz,WIAG-Inst-DIOCGatz-016-001
9,WIAG-Pers-EPISCGatz-05263-001,82446,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,119063085,009-01109-001,Q656048,NaN,NaN,left_only,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001


In [96]:
missing_institution_diocese_on_factgrid_df = dioc_joined_df[dioc_joined_df['fg_diocese_id'].isna()]

In [97]:
# missing_institution_diocese_on_factgrid_df.drop(['_merge'], axis=1, inplace=True)

In [98]:
# missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df.person_id.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]

## Create a csv file with missing ids for institution or diocese

In [99]:
# null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df['institution_id'].isnull()][missing_institution_diocese_on_factgrid_df['diocese_id'].isnull()]
null_entries = missing_institution_diocese_on_factgrid_df[missing_institution_diocese_on_factgrid_df['name'] != 'Kardinal'].sort_values('person_id')
null_entries = null_entries[null_entries['role_group'] != 'Kurienamt']
null_entries = null_entries[null_entries['role_group'] != 'Papst']
null_entries = null_entries.sort_values(['diocese', 'institution'])

# 4312 to 7979 should be removed
null_entries = null_entries[null_entries.person_id.str.contains('[0-7][0-9]{4}-[0-9]{3}$', regex=True)]
null_entries.to_csv(f'missing_amter-{today_string}_Bamberg.csv', sep=';')
null_entries

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,_merge,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id


# (Optionally) create the entries here

In [100]:
# TODO: sent ouput to file instead of stdout

# for _, row in null_entries[null_entries['diocese'].notnull()][['diocese', 'diocese_id']].iterrows():
#     print("CREATE")
#     print('\t'.join(['LAST', 'Lde', f'"{row["diocese"]}"']))
#     print('\t'.join(['LAST', 'P2', 'Q153166']))

#     print('\t'.join(['LAST', 'P131', 'Q153178']))
#     if type(row['diocese_id']) == str:
#         print('\t'.join(['LAST', 'P601', f'"{row["diocese_id"]}"']))

In [101]:
# missing_institution_diocese_on_factgrid_df.to_csv('no_institution_or_dioc_on_fg_bamberg.csv', sep=';')

### To continue below your csv files should be empty. The check below ensures that.

In [102]:
assert len(null_entries) == 0, "Create the entries on factgrid before continuing."

# Add factgrid information

## Add institution factgrid id

In [103]:
institution_joined_df = role_all_df.merge(
    factgrid_institution_df, how='left', left_on='institution_id', right_on='fg_gsn_id', suffixes=('_wiag', '_institute_fg')
)
print(len(institution_joined_df))
institution_joined_df.head()

2798


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,date_begin,date_end,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,1016,1047,1016150,120912619,062-01186-001,Q653844,NaN,NaN
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,(1014 ?) 1015,1019,1019150,136969143,059-00710-001,Q652691,NaN,NaN
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753,Q400557,3502.0
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,1020,NaN,1020150,NaN,NaN,Q728753,Q400530,3492.0
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,1057,1065,1057150,118699334,006-00007-001,Q652702,NaN,NaN


In [104]:
assert len(role_all_df) == len(institution_joined_df), "There may be duplicates on factgrid"

## Add diocese factgrid id

In [105]:
dioc_joined_df = join_dioceses(institution_joined_df)
print(len(dioc_joined_df))
dioc_joined_df.head()

2798


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,date_sort_key,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,1016150,120912619,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,1019150,136969143,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,1020150,NaN,NaN,Q728753,Q400557,3502.0,None,None,None,None
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,1020150,NaN,NaN,Q728753,Q400530,3492.0,None,None,None,None
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,1057150,118699334,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001


## Add role factgrid id
Note: This role does not include the institution information. ie, it adds factgrid ids for roles like 'archbishop' and not 'archbishop of trier'

The part of the script below could be used to create quickstatements for career statements.

In [106]:
print(len(wiag_roles_df))
wiag_roles_df.head()
#wiag_roles_df[wiag_roles_df['name'] == '']

102


,id,note,name,comment,gs_reg_id,generic_term,plural,definition,role_gp_fg_id,gender,lang,role_group_id,r_id,item_id,r_note,authority_id,role_fg_id,r_comment
0,3,NaN,Administrator des Bistums,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33.0,1298102,3,NaN,42,Q902170,NaN
1,4,NaN,Administrator des Erzbistums,NaN,NaN,NaN,NaN,NaN,Q648236,männlich,de,33.0,1298104,4,NaN,42,Q902170,NaN
2,5,NaN,Administrator des Fürstbischofs,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33.0,1298982,5,NaN,42,Q902170,NaN
3,9,NaN,Apostolischer Administrator,NaN,NaN,NaN,NaN,Ist ein Bischofsstuhl vakant oder bestehen ern...,Q648236,männlich,de,33.0,1298980,9,NaN,42,Q38958,NaN
4,13,NaN,Bischofsadministrator,NaN,NaN,NaN,NaN,Ein Bistumsadministrator (auch: Diözesanadmini...,Q648236,männlich,de,33.0,1298106,13,NaN,42,Q902170,NaN


In [107]:
joined_df = dioc_joined_df.merge(
    wiag_roles_df[['name', 'role_fg_id']], how='left', left_on='name', right_on='name', suffixes=('_wiag', '_institute_fg')
)
print(len(joined_df))
joined_df.head()

2807


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,120912619,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001,Q172539
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,136969143,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001,Q38809
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,NaN,NaN,Q728753,Q400557,3502.0,None,None,None,None,Q38837
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,Q728753,Q400530,3492.0,None,None,None,None,Q38837
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,118699334,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38809


### Ignore all Kanonikatsbewerber and Vikariatsbewerber offices


In [108]:
joined_df = joined_df[~joined_df['name'].isin(
    ['Vikariatsbewerber', 
     'Kanonikatsbewerber']
)]

## The output below should be empty. The cell after the next one will throw an error if it's not the case

In [110]:
joined_df[joined_df['role_fg_id'].isna()][['person_id', 'name']]
# joined_df[joined_df['role_fg_id'].isna()]

,person_id,name


In [111]:
assert len(joined_df[joined_df['role_fg_id'].isna()]) == 0, "Missing roles with factgrid id in wiag database"

## Check people with missing factgrid entries or missing factgrid ids in wiag

In [112]:
joined_df[joined_df['FactGrid'].isna()]

,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GND,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id
65,WIAG-Pers-CANON-80982-001,56198,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,NaN,007-01739-001,NaN,Q400530,3492.0,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38837
637,WIAG-Pers-CANON-25882-001,140432,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,13948941X,012-01202-001,NaN,Q400557,3502.0,None,None,None,None,Q38837
638,WIAG-Pers-CANON-25882-001,140434,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,13948941X,012-01202-001,NaN,Q400530,3492.0,None,None,None,None,Q38837
639,WIAG-Pers-CANON-25882-001,140436,Domdekan,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Würzburg,3502.0,NaN,NaN,...,13948941X,012-01202-001,NaN,Q400557,3502.0,None,None,None,None,Q38836
640,WIAG-Pers-CANON-25882-001,140438,Generalvikar,Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Würzburg,WIAG-Inst-DIOCGatz-033-001,...,13948941X,012-01202-001,NaN,NaN,NaN,Q153247,Bistum Würzburg,Diözese Würzburg,WIAG-Inst-DIOCGatz-033-001,Q39117
1149,WIAG-Pers-CANON-48974-001,139834,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q38837
1228,WIAG-Pers-CANON-45781-001,139446,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q38837
1229,WIAG-Pers-CANON-45781-001,139448,Domcellerar,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q893490
1230,WIAG-Pers-CANON-45781-001,139450,Domkustos,Leitungsamt Domstift,Dignitary of a cathedral chapter,Q648232,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,NaN,NaN,Q400530,3492.0,None,None,None,None,Q902176
1231,WIAG-Pers-CANON-45781-001,139452,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Augsburg,3498.0,NaN,NaN,...,NaN,NaN,NaN,Q898020,3498.0,None,None,None,None,Q38837


In [113]:
joined_df[joined_df['FactGrid'].isna()]['person_id'].unique()

array(['WIAG-Pers-CANON-80982-001', 'WIAG-Pers-CANON-25882-001',
       'WIAG-Pers-CANON-48974-001', 'WIAG-Pers-CANON-45781-001',
       'WIAG-Pers-CANON-48993-001', 'WIAG-Pers-CANON-24731-001',
       'WIAG-Pers-CANON-25122-001', 'WIAG-Pers-CANON-49126-001',
       'WIAG-Pers-CANON-14941-001', 'WIAG-Pers-CANON-14944-001',
       'WIAG-Pers-CANON-15005-001', 'WIAG-Pers-CANON-26451-001',
       'WIAG-Pers-CANON-15070-001'], dtype=object)

## (Optinally) generate the quickstatements for creating the persons here

In [114]:
joined_df[joined_df['FactGrid'].isna()].to_csv(f'missing_factgrid-{today_string}_Bamberg.csv', sep=';')

## The cell below will throw an error if there are entries on Factgrid but not on Wiag

In [115]:
assert joined_df[joined_df['FactGrid'].isna()].empty, "There are missing persons on factgrid"

AssertionError: There are missing persons on factgrid

## WARNING: the code section below ignores all entries absent on factgrid

In [116]:
joined_df = joined_df[~joined_df['FactGrid'].isna()]

In [117]:
factgrid_inst_roles_df

,fg_inst_role_id,inst_role
0,Q172316,Generalvikar der Diözese Trento
1,Q172317,Generalvikar der Diözese Utrecht
2,Q172318,Generalvikar der Diözese Vienna
3,Q172319,Generalvikar der Diözese Lausanne
4,Q172320,Generalvikar der Diözese Pomesanien
...,...,...
4338,Q452118,Schauspieler/Schauspielerin des Künstlertheaters
4339,Q452119,Schauspieler/in des Belvárosi-Theaters
4340,Q452120,Schauspieler/Schauspielerin des Pester Theaters
4341,Q452121,Schauspieler/Schauspielerin des Theaters der U...


## Add factgrid ids for roles
Note: this role has information of the institution as well

In [118]:
# add factgrid ids for roles
found = 0
data_dict = {}
not_found = []
dupl = {}
for i, (name, inst, dioc) in joined_df[['name', 'institution', 'diocese']].iterrows():
    if name == "Kardinal":
        # this is okay
        # manually add qid for kardinals Q254893
        data_dict[i] = "Q254893"
        continue
    search_res = pd.DataFrame()
    if pd.isna(inst):
    # if name in ["Bischof", "Fürstbischof", "Ernannter Bischof", "Erzbischof"]:
        if pd.isna(dioc):
            print(i, name, inst, dioc)
        if name not in ["Archidiakon", "Koadjutor"]:
            dioc = dioc.lstrip('Bistum').lstrip('Erzbistum').lstrip('Patriarchat').lstrip()
        if name == "Fürstbischof" and dioc in ["Passau", "Straßburg"]:
            name = "Bischof"    
        search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['inst_role'].str.contains(f"^{name}.*{dioc}")]
        if name == "Erzbischof" and dioc == "Salzburg":
            # will be merged in later
            search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['fg_inst_role_id'] == 'Q172567']
    else:
        name = name.replace('Domkanoniker', 'Domherr')
        search_res = factgrid_inst_roles_df[factgrid_inst_roles_df['inst_role'] == f"{name} {inst}"]
    if len(search_res) == 1:
        found += 1
        data_dict[i] = search_res['fg_inst_role_id'].values[0]
    elif len(search_res) >= 2:
        # print("+" * 10)
        print(name, inst, dioc)
        print(search_res)
        print()
        dupl[i] = (name, inst, dioc, search_res)
    elif len(search_res) == 0:
        not_found.append((name, inst, dioc))
        print(name, inst, dioc)
print("===Summary of roles===")
print("Found:", found, "Duplicates:", len(dupl), "Not found:", len(not_found))

Propst Kollegiatstift St. Nikolaus, Spalt nan
Domsenior Domstift Würzburg nan
Cellerar Domstift Eichstätt nan
Propst Benediktinerkloster, später Kollegiatstift St. Cyriakus, Wiesensteig nan
Domscholaster Domstift Eichstätt nan
Domcellerar Domstift Augsburg nan
Kapitularvikar nan Würzburg
Propst Kollegiatstift Unsere Liebe Frau, Eichstätt nan
===Summary of roles===
Found: 2742 Duplicates: 0 Not found: 8


In [119]:
final_joined_df = joined_df.merge(pd.Series(data_dict).rename('fg_inst_role_id'), left_index=True, right_index=True)
print(len(final_joined_df))
final_joined_df.head()

2743


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id,fg_inst_role_id
0,WIAG-Pers-EPISCGatz-05072-001,5154,Erzbischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Erzbistum Trier,WIAG-Inst-DIOCGatz-030-001,...,062-01186-001,Q653844,NaN,NaN,Q153244,Erzbistum Trier,Bistum Trier,WIAG-Inst-DIOCGatz-030-001,Q172539,Q172572
1,WIAG-Pers-EPISCGatz-04050-001,3770,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Eichstätt,WIAG-Inst-DIOCGatz-006-001,...,059-00710-001,Q652691,NaN,NaN,Q153220,Bistum Eichstätt,Diözese Eichstätt,WIAG-Inst-DIOCGatz-006-001,Q38809,Q172630
2,WIAG-Pers-CANON-24965-001,22325,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Würzburg,3502.0,NaN,NaN,...,NaN,Q728753,Q400557,3502.0,None,None,None,None,Q38837,Q390632
3,WIAG-Pers-CANON-24965-001,31418,Domherr,Amt Domstift,Cleric of a cathedral chapter,Q648226,Domstift Bamberg,3492.0,NaN,NaN,...,NaN,Q728753,Q400530,3492.0,None,None,None,None,Q38837,Q400601
4,WIAG-Pers-EPISCGatz-03869-001,3586,Bischof,Oberstes Leitungsamt Diözese,Head of an (arch)diocese,Q648236,NaN,NaN,Bistum Bamberg,WIAG-Inst-DIOCGatz-002-001,...,006-00007-001,Q652702,NaN,NaN,Q153216,Bistum Bamberg,Diözese Bamberg,WIAG-Inst-DIOCGatz-002-001,Q38809,Q195266


## Parse begin and end date from the wiag data

In [137]:
# https://database.factgrid.de/query/embed.html#SELECT%20%3FPropertyLabel%20%3FProperty%20%3FPropertyDescription%20%3Freciprocal%20%3FreciprocalLabel%20%3Fexample%20%3Fuseful_statements%20%3Fwd%20WHERE%20%7B%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%20%20%3FProperty%20wdt%3AP8%20wd%3AQ77483.%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP364%20%3Fexample.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP86%20%3Freciprocal.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP343%20%3Fwd.%20%7D%0A%20%20OPTIONAL%20%7B%20%3FProperty%20wdt%3AP310%20%3Fuseful_statements.%20%7D%0A%7D%0AORDER%20BY%20%3FPropertyLabel

def format_datetime(entry: datetime, resolution):
    julian_ending = "/J" if entry.year < 1582 else ""
    ret_val =  f"+{entry.isoformat()}Z/{resolution}" + julian_ending
    if resolution <= 9:
        ret_val = ret_val.replace(f"{entry.year}-01-01", f"{entry.year}-00-00", 1)
    return ret_val

def date_parsing(date_string: str, end=False, only_date=False):
    return_property = "P106" if only_date else "P50" if end else "P49"
    qualifier = None
    entry = None
    resolution = 7
    string_precision_qualifier_clause = ("P467" if only_date else "P788" if end else "P787") + f'\t"{date_string}"'
    exact_precision_qualifier = ("P467" if only_date else "P786" if end else "P785")

    if pd.isna(date_string) or date_string == '?':
        return tuple()
    
    if matches := re.match(r'frühestens (\d{3,4})', date_string):
        return_property = "P41" if only_date else "P1125" if end else "P1126"
        entry = datetime(int(matches.group(1)), 1, 1)
        resolution = 9
    
    elif matches := re.match(r'(kurz )?vor (\d{3,4})', date_string):
        return_property = "P43" if only_date else "P1123" if end else "P1124"
        if matches.group(1):
            qualifier = string_precision_qualifier_clause
        entry = datetime(int(matches.group(2)), 1, 1)
        resolution = 9
        
    elif matches := re.match(r'(kurz )?nach (\d{3,4})', date_string):
        return_property = "P41" if only_date else "P1125" if end else "P1126"
        if matches.group(1):
            qualifier = string_precision_qualifier_clause
        entry = datetime(int(matches.group(2)), 1, 1)
        resolution = 9
        
    elif matches := re.match(r'(\d{1,2})\. Jahrhundert', date_string):
        century = int(matches.group(1))
        entry = datetime(100 * (century), 1, 1)
    
    elif matches := re.match(r'(\d)\. Hälfte (des )?(\d{1,2})\. (Jhs\.|Jahrhunderts|Jahrhundert)', date_string):
        half = int(matches.group(1))
        year = int(matches.group(3)) - 1
        latest_year   = year * 100 + (half * 50)
        earliest_year = latest_year - 50 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = string_precision_qualifier_clause
    
    elif matches := re.match(r'(\w+) Viertel des (\d{1,2})\. Jhs\.', date_string):
        number_map = {
            "erstes":  1,
            "zweites": 2,
            "drittes": 3,
            "viertes": 4,
        }
        quarter = number_map[matches.group(1)]
        year    = int(matches.group(2)) - 1
        latest_year   = year * 100 + (quarter * 25)
        earliest_year = latest_year - 25 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'frühes (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1)) - 1
        latest_year   = year * 100 + 20
        earliest_year = latest_year - 20 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'spätes (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1))
        latest_year   = year * 100
        earliest_year = latest_year - 20 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'(Anfang|Mitte|Ende) (\d{1,2})\. Jh\.', date_string):
        number_map = {
            "Anfang":  1,
            "Mitte": 2,
            "Ende": 3,
        }
        third = number_map[matches.group(1)]
        year = int(matches.group(2)) - 1
        latest_year   = year * 100 + (third * 33)
        earliest_year = latest_year - 33 + 1
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'(ca\.|um) (\d{3,4})', date_string):
        year = int(matches.group(2))
        latest_year   = year + 5
        earliest_year = latest_year - 10
        entry = datetime((earliest_year + latest_year) // 2, 1, 1)
        resolution = 9
        qualifier = exact_precision_qualifier + "\tQ10"

    elif matches := re.match(r'(\d{3,4})er Jahre', date_string):
        year = int(matches.group(1))
        latest_year   = year + 10
        earliest_year = latest_year - 10 + 1
        entry = datetime(year, 1, 1)
        resolution = 8
    
    elif matches := re.match(r'Wende zum (\d{1,2})\. Jh\.', date_string):
        year = int(matches.group(1)) - 1
        latest_year   = year * 100 + 10
        earliest_year = latest_year - 20 + 1
        entry = datetime(latest_year, 1, 1)
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'Anfang der (\d{3,4})er Jahre', date_string):
        year = int(matches.group(1))
        latest_year   = year + 3
        earliest_year = year
        entry = datetime(year, 1, 1)
        resolution = 8
        qualifier = string_precision_qualifier_clause

    elif matches := re.match(r'\((\d{3,4}) \?\) (\d{3,4})', date_string):
        year1 = int(matches.group(1))
        year2 = int(matches.group(2))
        entry = datetime(year2, 1, 1)
        resolution = 9
        qualifier = string_precision_qualifier_clause
    
    elif matches := re.match(r'(\d{3,4})/(\d{3,4})', date_string):
        year1 = int(matches.group(1))
        year2 = int(matches.group(2))

        if year2 - year1 == 1:
            # check for consecutive years
            qualifier = exact_precision_qualifier + "\tQ912616"
        entry = datetime(year1, 1, 1)
        resolution = 9

    elif matches := re.match(r'(\d{3,4})\?', date_string):
        year = int(matches.group(1))
        entry = datetime(year, 1, 1)
        resolution = 9
        qualifier = exact_precision_qualifier + f'\t"{date_string}"'
    
    elif matches := re.match(r'(\d{3,4})', date_string):
        year = int(matches.group(1))
        entry = datetime(year, 1, 1)
        resolution = 9
    else:
        print(date_string)
        raise Exception("Couldn't parse date")
        
    if qualifier:
        return (return_property, format_datetime(entry, resolution), qualifier)
    else:
        return (return_property, format_datetime(entry, resolution))

tests = {
    "1205": "+1205-00-00T00:00:00Z/9/J",
    "12. Jahrhundert": "+1200-00-00T00:00:00Z/7/J",
    "1. Hälfte des 12. Jhs.": "+1125-00-00T00:00:00Z/7/J",
    "2. Hälfte des 12. Jhs.": "+1175-00-00T00:00:00Z/7/J",
    "erstes Viertel des 12. Jhs.": "+1113-00-00T00:00:00Z/7/J",
    "zweites Viertel des 12. Jhs.": "+1138-00-00T00:00:00Z/7/J",
    "drittes Viertel des 12. Jhs.": "+1163-00-00T00:00:00Z/7/J",
    "viertes Viertel des 12. Jhs.": "+1188-00-00T00:00:00Z/7/J",
    "frühes 12. Jh.": "+1110-00-00T00:00:00Z/7/J",
    "spätes 12. Jh.": "+1190-00-00T00:00:00Z/7/J",
    "Anfang 12. Jh.": "+1117-00-00T00:00:00Z/7/J",
    "Mitte 12. Jh.": "+1150-00-00T00:00:00Z/7/J",
    "Ende 12. Jh.": "+1183-00-00T00:00:00Z/7/J",
    "ca. 1050": "+1050-00-00T00:00:00Z/9/J",
    "um 1050": "+1050-00-00T00:00:00Z/9/J",
    "1230er Jahre": "+1230-00-00T00:00:00Z/8/J",
    "Wende zum 12. Jh.": "+1110-00-00T00:00:00Z/7/J",
    "Anfang der 1480er Jahre": "+1480-00-00T00:00:00Z/8/J",
    "frühestens 1342": "+1342-00-00T00:00:00Z/9/J",
    "vor 1230": "+1230-00-00T00:00:00Z/9/J",
    "nach 1230": "+1230-00-00T00:00:00Z/9/J",
    "kurz vor 1200": "+1200-00-00T00:00:00Z/9/J",
    "kurz nach 1200": "+1200-00-00T00:00:00Z/9/J",
    "1164/1165": "+1164-00-00T00:00:00Z/9/J",
    "1164/1177": "+1164-00-00T00:00:00Z/9/J",
}

for key, value in tests.items():
    retval = date_parsing(key)[1]
    assert retval == value, f"{key}: Returned {retval} instead of {value}"


## Reconcile office data with factgrid

In [138]:
#https://database.factgrid.de/wiki/Special:EntityData/Q515.json

## Generate quickstatements for offices

In [139]:
with open(f'quickstatments_{today_string}.qs', 'w') as file:
    for _, row in final_joined_df.iterrows():
        try:
            date_clauses = ()
            if pd.isna(row['date_begin']):
                date_clauses = date_parsing(row['date_begin'], only_date=True)
            elif pd.isna(row['date_end']):
                date_clauses = date_parsing(row['date_begin'], only_date=True)
            else:
                date_clauses = (*date_parsing(row['date_begin']), *date_parsing(row['date_end'], end=True))
            file.write('\t'.join([
                row['FactGrid'], 
                'P165', 
                row['fg_inst_role_id'],
                'S601', 
                '"' + row['person_id'] + '"',
                *date_clauses,
            ]) + '\n')
        except Exception as e:
            print(traceback.format_exc())
            print(row)

In [132]:
final_joined_df[final_joined_df['name'] == 'Kanoniker'][final_joined_df['person_id'] == 'WIAG-Pers-EPISCGatz-10347-001']

C:\Users\khan32\AppData\Local\Temp\ipykernel_7824\3387915465.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_joined_df[final_joined_df['name'] == 'Kanoniker'][final_joined_df['person_id'] == 'WIAG-Pers-EPISCGatz-10347-001']


,person_id,id,name,role_group,role_group_en,role_group_fq_id,institution,institution_id,diocese,diocese_id,...,GSN,FactGrid,fg_institution_id,fg_gsn_id,fg_diocese_id,dioc_label,dioc_alt,dioc_wiag_id,role_fg_id,fg_inst_role_id
2434,WIAG-Pers-EPISCGatz-10347-001,137596,Kanoniker,Amt Stift,Affiliate of a collegiate church or convent,Q648228,"Kollegiatstift St. Viktor, Mainz",3249.0,NaN,NaN,...,082-01197-001,Q654987,Q400416,3249.0,None,None,None,None,Q38823,Q907996
2436,WIAG-Pers-EPISCGatz-10347-001,137600,Kanoniker,Amt Stift,Affiliate of a collegiate church or convent,Q648228,"Kollegiatstift St. Peter und St. Alexander, As...",1016.0,NaN,NaN,...,082-01197-001,Q654987,Q400401,1016.0,None,None,None,None,Q38823,Q400574
2439,WIAG-Pers-EPISCGatz-10347-001,137606,Kanoniker,Amt Stift,Affiliate of a collegiate church or convent,Q648228,"Benediktinerabtei, dann Kollegiatstift St. Alb...",3241.0,NaN,NaN,...,082-01197-001,Q654987,Q400526,3241.0,None,None,None,None,Q38823,Q907994


In [45]:
# all_roles = set()

# def add_possible_list(a_set: set, element):
#     if type(element) == list:
#         # add all items in list
#         for item in element:
#             a_set.add(item)
#     else:
#         # simply add the given element
#         a_set.add(element)

# for summary in missing_on_factgrid_df['summary_offices'].tolist():
#     print(summary)
#     offices = re.split(r'(\d+),', summary)
#     offices = [office.lstrip() for office in offices]
#     new_offices = offices
#     for index, office in enumerate(offices):
#         if re.match('\d+', office):
#             new_offices[index - 1] += office
#             new_offices.pop(index)
#     print(new_offices)
#     for office in new_offices:
#         for office_name in re.match(r'\w+(, \w+)*', office).group().split(','):
#             office_name = office_name.strip()
#             print(office_name)
#             all_roles.add(office_name)
#     print()

# print('#'*10)
# print(all_roles)

# url = 'https://database.factgrid.de/sparql'
# query = (
#     f"""SELECT DISTINCT ?item ?label
# WHERE
# {{
#   ?item wdt:P2 wd:Q37073;
#         rdfs:label ?label.
#   FILTER(LANG(?label) = "de").
#   FILTER REGEX (?label, "({'|'.join(list(all_roles))})$").
# }}
# ORDER BY ?label  
# """
# )
# print(query)
# # SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

# # make request: 
# r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
# data = r.json()
# factgrid_roles_df = pd.json_normalize(data['results']['bindings'])

# len(factgrid_roles_df)

# list(all_roles)